In [4]:
# Import the SPICE module
import spiceypy

In [5]:
# We want to determine the position of our home planet with respect to the Sun.
# The datetime shall be set as "today" (midnight). SPICE requires the
# Ephemeris Time (ET); thus, we need to convert a UTC datetime string to ET.

import datetime

# get today's date
DATE_TODAY = datetime.datetime.today()

# convert the datetime to a string, replacing the time with midnight
DATE_TODAY = DATE_TODAY.strftime('%Y-%m-%dT00:00:00')

# convert the utc midnight string to the corresponding ET
spiceypy.furnsh('../kernels/lsk/naif0012.tls') #<-- This is needed.
ET_TODAY_MIDNIGHT = spiceypy.utc2et(DATE_TODAY)

In [8]:
# To compute now the position and velocity (so called state) of the Earth
# with respect to the Sun, we use the following function to determine the
# state vector and the so called light time (travel time of the light between 
# the Sun and our home planet). Positions are always given in km, velocities 
# in km/s and times in seconds

#First we load the kernel for positional information first:
spiceypy.furnsh('../kernels/spk/de432s.bsp')

# targ : Object that shall be pointed at (399 := Earth)
# et : The ET of the computation (Set for today)
# ref : The reference frame. Here, it is ECLIPJ2000 (the ecliptic plane of the Earth)
# obs :  The observer respectively the center of our state vector computation (10 := Sun)
EARTH_STATE_WRT_SUN, EARTH_SUN_LT = spiceypy.spkgeo(targ=399, \
                                                    et=ET_TODAY_MIDNIGHT, \
                                                    ref='ECLIPJ2000', \
                                                    obs=10)

In [19]:
#The first 3 values are the x, y, z components in km. 
#The last 3 values are the corresponding velocity components in km/s.
print(EARTH_STATE_WRT_SUN)

[-1.47245097e+08  2.04697645e+07 -5.72276764e+02 -4.57590066e+00
 -2.96141750e+01  9.30806649e-04]


In [20]:
# Is the one-way light time from the observing body
# to the geometric position of the target body
# in seconds at the specified epoch.
# It should be around 8mins
print(EARTH_SUN_LT/60)

8.264668887624136


In [11]:
# The (Euclidean) distance should be around 1 AU. Why "around"? Well the Earth
# revolves the Sun in a slightly non-perfect circle (elliptic orbit). First, 
# we compute the distance in km.
import math
EARTH_SUN_DISTANCE = math.sqrt(EARTH_STATE_WRT_SUN[0]**2.0 \
                             + EARTH_STATE_WRT_SUN[1]**2.0 \
                             + EARTH_STATE_WRT_SUN[2]**2.0)

# Convert the distance in astronomical units (1 AU)
# Instead of searching for the "most recent" value, we use the default value
# in SPICE. This way, we can easily compare our results with the results of 
# others.
EARTH_SUN_DISTANCE_AU = spiceypy.convrt(EARTH_SUN_DISTANCE, 'km', 'AU')

# Cool, it works!
print('Current distance between the Earth and the Sun in AU:', \
      EARTH_SUN_DISTANCE_AU)

Current distance between the Earth and the Sun in AU: 0.9937382357968857


In [12]:
#Lets comute the oribital speed of the Earth in km/s
# First, we compute the actual orbital speed of the Earth around the Sun
EARTH_ORB_SPEED_WRT_SUN = math.sqrt(EARTH_STATE_WRT_SUN[3]**2.0 \
                                  + EARTH_STATE_WRT_SUN[4]**2.0 \
                                  + EARTH_STATE_WRT_SUN[5]**2.0)

# It's around 30 km/s
print('Current orbital speed of the Earth around the Sun in km/s:', \
      EARTH_ORB_SPEED_WRT_SUN)

Current orbital speed of the Earth around the Sun in km/s: 29.96561742728447


Now we get the theoretical earth orbital speed:

In [14]:
# Now let's compute the theoretical expectation. First, we load a pck file
# that contain miscellanoeus information, like the G*M values for different
# objects

# First, load the kernel
spiceypy.furnsh('../kernels/pck/gm_de431.tpc')
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

# Now compute the orbital speed
V_ORB_FUNC = lambda gm, r: math.sqrt(gm/r)
EARTH_ORB_SPEED_WRT_SUN_THEORY = V_ORB_FUNC(GM_SUN[0], EARTH_SUN_DISTANCE)

# Print the result
print('Theoretical orbital speed of the Earth around the Sun in km/s:', \
      EARTH_ORB_SPEED_WRT_SUN_THEORY)

Theoretical orbital speed of the Earth around the Sun in km/s: 29.87838444261713
